In [0]:
%sql
-- Ingestão Bronze:
-- Azure Databricks + Unity Catalog + Volumes com as duas ingestões separadas.
-- Obs: Catálogo Criado e Schema Bronze também em Unity Catalog. No arquivo '00.config'
-- Criação de pastas volume no 'databricks'

CREATE VOLUME IF NOT EXISTS ampev.bronze.landings

In [0]:
# Ingestão Bronze:
# Azure Databricks + Unity Catalog + Volumes com as duas ingestões separadas.

BASE = "/Volumes/ampev/bronze/landings"  

dbutils.fs.mkdirs(BASE)
dbutils.fs.mkdirs(f"{BASE}/estabelecimentos")
dbutils.fs.mkdirs(f"{BASE}/pedidos")

display(dbutils.fs.ls(BASE))


In [0]:
#  “Bootstrap” controlado: pasta samples

BASE = "/Volumes/ampev/bronze/landings"
SAMPLES = f"{BASE}/samples"

dbutils.fs.mkdirs(SAMPLES)
dbutils.fs.mkdirs(f"{SAMPLES}/estabelecimentos")
dbutils.fs.mkdirs(f"{SAMPLES}/pedidos")

display(dbutils.fs.ls(BASE))




In [0]:
# Leitura dos samples :

df = spark.read.format("csv").load(f"{SAMPLES}/estabelecimentos/estabelecimentos_sample.csv", header=True, inferSchema=True)
display(df)
#
# Leitura dos samples:

df_2 = spark.read.format("csv").load(f"{SAMPLES}/pedidos/pedidos_sample.csv", header=True, inferSchema=True)
display(df_2)

In [0]:
# Inferir schemas apartir dos samples:

schema_estab = (
  spark.read
    .option("header","true")
    .option("sep",",")
    .csv(f"{SAMPLES}/estabelecimentos/*.csv")
    .schema
)

schema_ped = (
  spark.read
    .option("header","true")
    .option("sep",",")
    .csv(f"{SAMPLES}/pedidos/*.csv")
    .schema
)

print(f"Schema estabelecimentos: {schema_estab}")
print(f"Schema pedidos: {schema_ped}")



In [0]:
# Auto Loader separado por arquivos: estabelecimentos.csv

LANDING = "/Volumes/ampev/bronze/landings"

df_estab = (spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "csv")
  .option("cloudFiles.schemaLocation", f"{LANDING}/_schemas/estabelecimentos")
  .option("header", "true")
  .schema(schema_estab)
  .load(f"{LANDING}/estabelecimentos")
)
   

  



In [0]:
# Auto Loader separado: pedidos

df_ped = (spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "csv")
  .option("cloudFiles.schemaLocation", f"{LANDING}/_schemas/pedidos")
  .option("header", "true")
  .schema(schema_ped)
  .load(f"{LANDING}/pedidos")
)




Obs: O Landing pode ficar vazia, não depende de arquivo real e não gera erro 'CF_EMPTY_DIR_FOR_SCHEMA_INFERENCE'
O Schema fica congelado (controle total) e evita inferência errada quando chegar arquivo grande.

Os diretórios de 'checkpointLocation' só são criados quando o streaming realmente inicia e faz o primeiro commit.
Ele nasce quando acontece o primeiro micro-batch com dados.
Fluxo normal: O arquivo cai na pasta, Stream inicia, Auto Loader detecta arquivo e Processa micro-batch.
Assim cria a pasta de checkpoint, metadata e tabela Delta.

In [0]:
# Controle de arquivos para Streaming:
from pyspark.sql.functions import current_timestamp, col
def has_files(path: str) -> bool:
    try:
        files = dbutils.fs.ls(path)
        # ignora pastas internas como _schemas/_checkpoints
        data_files = [f for f in files if not f.name.startswith("_")]
        return len(data_files) > 0
    except Exception:
        return False

# Conditional para iniciar stream do arquivo estabelecimentos:
#     
src_estab = f"{LANDING}/estabelecimentos"

if has_files(src_estab):
    print("Arquivos encontrados para estabelecimentos. Iniciando stream...")

    from pyspark.sql.functions import current_timestamp, input_file_name

    df_estab_out = (df_estab
      .withColumn("_ingest_ts", current_timestamp())
      .withColumn("_source_file", col("_metadata.file_path"))
    )

    q_estab = (df_estab_out.writeStream
      .format("delta")
      .option("checkpointLocation", f"{LANDING}/_checkpoints/estabelecimentos")
      .trigger(availableNow=True)
      .toTable("ampev.bronze.estabelecimentos")
    )

else:
    print("Nenhum arquivo encontrado para estabelecimentos. Stream não iniciado.")

# Conditional para iniciar stream do arquivo pedidos:
#     
src_ped = f"{LANDING}/pedidos"

if has_files(src_ped):
    print("Arquivos encontrados para pedidos. Iniciando stream...")

    from pyspark.sql.functions import current_timestamp, input_file_name

    df_ped_out = (df_ped
      .withColumn("_ingest_ts", current_timestamp())
      .withColumn("_source_file", col("_metadata.file_path"))
    )

    q_ped = (df_ped_out.writeStream
      .format("delta")
      .option("checkpointLocation", f"{LANDING}/_checkpoints/pedidos")
      .trigger(availableNow=True)
      .toTable("ampev.bronze.pedidos")
    )

else:
    print("Nenhum arquivo encontrado para pedidos. Stream não iniciado.")



Dessa forma isso garante em não gerar erro de inferência, não cria checkpoint vazio, não inicia stream desnecessário.
Assim, funciona perfeitamente com Job agendado e continua incremental quando arquivos chegarem.

Recomandações para produção:

- Agende esse notebook como Job (ex: a cada 5 minutos)
- Se não houver arquivos → termina em segundos
- Se houver arquivos → processa backlog e encerra

E o checkpoint garante que nunca reprocessa o mesmo arquivo.